In [10]:
import time
from datetime import datetime
from fyers_apiv3 import fyersModel
import os

symbol_script = 'NSE:NIFTYBANK-INDEX'

current_dir1 = os.path.dirname(os.getcwd())
current_dir2 = os.path.dirname(current_dir1)
file_path = os.path.join(current_dir2, 'access.txt')

def get_access_token(file_path):
    with open(file_path, "r") as file:
        file_content = file.read()
    return file_content

client_id='6MS9YXZMMR-100'
fyers = fyersModel. FyersModel (token= get_access_token(file_path), log_path=os.getcwd(), client_id=client_id)

In [11]:
data = {"symbol":"NSE:SBIN-EQ", "resolution":"15", "date_format": "0","range_from": "1704947400", "range_to": "1704961800", "cont_flag": "1"}
sdata= fyers.history(data)
sdata

{'candles': [[1704947400, 623.8, 624, 621.2, 621.95, 761222],
  [1704948300, 622, 624.95, 621.95, 624.7, 445417],
  [1704949200, 624.65, 626, 623.35, 623.85, 477814],
  [1704950100, 623.95, 624.75, 621.7, 623.3, 400730],
  [1704951000, 623.4, 623.55, 622.2, 622.95, 246282],
  [1704951900, 623, 623.65, 621.85, 621.85, 600595],
  [1704952800, 622, 622.6, 621.3, 622.25, 530957],
  [1704953700, 622.1, 622.4, 620.7, 621.15, 682995],
  [1704954600, 621.2, 621.75, 620.55, 620.7, 461822],
  [1704955500, 620.65, 622.55, 620.5, 622, 635851],
  [1704956400, 622, 623, 621.6, 622.9, 336143],
  [1704957300, 623, 623.2, 621.7, 622.05, 438878],
  [1704958200, 622.15, 622.25, 621.15, 621.55, 393678],
  [1704959100, 621.55, 622.1, 621.15, 621.4, 265416],
  [1704960000, 621.4, 622.65, 621.15, 622.3, 330814],
  [1704960900, 622.3, 622.5, 621.1, 621.5, 457951],
  [1704961800, 621.5, 622.3, 618.65, 619.55, 1139898]],
 'code': 200,
 'message': '',
 's': 'ok'}

In [ ]:
def fetch_realtime_candles(symbol, time_frame, candle_numbers, fyers_client):
    # Calculate the resolution in seconds
    resolution_seconds = int(time_frame) * 60

    # Fetch the current epoch time
    current_epoch_time = int(time.time())

    # Calculate the start time for historical data
    start_epoch_time = current_epoch_time - (candle_numbers * resolution_seconds)

    # Prepare the data for the API call
    data = {
        "symbol": symbol,
        "resolution": time_frame,
        "date_format": "0",
        "range_from": str(start_epoch_time),
        "range_to": str(current_epoch_time),
        "cont_flag": "1"
    }

    # Make the history API call
    historical_data = fyers_client.history(data)

    # Process the historical data
    candlesticks = []
    for candle in historical_data['candles']:
        candle_obj = {
            'epoch_time': candle['timestamp'],
            'open': candle['open'],
            'high': candle['high'],
            'low': candle['low'],
            'close': candle['close'],
            'volume': candle['volume']
        }
        candlesticks.append(candle_obj)

    # Print or use the historical data as needed
    print("Historical Candlesticks:")
    print(candlesticks)

    # Continuously fetch new real-time data and append to the list
    while True:
        # Fetch real-time data
        real_time_data = fyers_client.get_realtime_data(symbol)

        # Process the real-time data
        new_candle = {
            'epoch_time': real_time_data['timestamp'],
            'open': real_time_data['open'],
            'high': real_time_data['high'],
            'low': real_time_data['low'],
            'close': real_time_data['close'],
            'volume': real_time_data['volume']
        }

        # Append the new candle to the list
        candlesticks.append(new_candle)

        # Print or use the new candle as needed
        print(f"New Candlestick: {new_candle}")

        # Wait for the next candle (adjust sleep time based on your time frame)
        time.sleep(resolution_seconds)

In [14]:
import datetime

def get_closest_past_epoch(time_frame_minutes):
    # Get the current time in GMT+5:30 (Indian Standard Time)
    current_time = datetime.datetime.utcnow() + datetime.timedelta(hours=5, minutes=30)

    # Extract the current hour and minute
    current_hour = current_time.hour
    current_minute = current_time.minute

    # Calculate the nearest multiple of time_frame_minutes in the past
    nearest_past_minute = (current_minute // time_frame_minutes) * time_frame_minutes
    nearest_past_time = current_time.replace(minute=nearest_past_minute, second=0, microsecond=0)

    # If the current time is before 9:15 AM, go back to the previous day
    if current_hour < 9 or (current_hour == 9 and current_minute < 15):
        nearest_past_time -= datetime.timedelta(days=1)

    # Convert the nearest past time to epoch time
    epoch_time = int(nearest_past_time.timestamp())

    return epoch_time

# Example usage:
time_frame_minutes = 1
epoch_time = get_closest_past_epoch(time_frame_minutes)
print(f"Closest past epoch time for a {time_frame_minutes}-minute time frame: {epoch_time}")


Closest past epoch time for a 30-minute time frame: 1704942000
